In [1]:
import azureml
from IPython.display import display, Markdown
from azureml.core import  Experiment, ScriptRunConfig, Workspace, RunConfiguration
from azureml.core.dataset import Dataset
from azureml.core.environment import Environment
from azureml.core.runconfig import DockerConfiguration


from model_drift import settings, helpers
# check core SDK version number
print("Azure ML SDK Version: ", azureml.core.VERSION)

Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azure-mgmt-resource 13.0.0 (d:\code\mlopsday2\.venv\lib\site-packages), Requirement.parse('azure-mgmt-resource<21.0.0,>=15.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azure-mgmt-resource 13.0.0 (d:\code\mlopsday2\.venv\lib\site-packages), Requirement.parse('azure-mgmt-resource<21.0.0,>=15.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azure-mgmt-resource 13.0.0 (d:\code\mlopsday2\.venv\lib\site-packages), Requirement.parse('azure-mgmt-resource<21.0.0,>=15.0.0'), {'azureml-core'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml

Azure ML SDK Version:  1.38.0


In [2]:
# Connect to workspace
ws = Workspace.from_config(settings.AZUREML_CONFIG)

In [3]:
# Name experiement
input_dataset_name="padchest"
experiment_name = 'generate-data-score'
model_name = "classifier-padchest-trained-chx-frontal-lime_train_07pvtv2y-047.ckpt"
env_name = "monitoring"

compute_target = "nc24-uswest2"

#Experiment
exp = Experiment(workspace=ws, name=experiment_name)

#Environment
environment_file = settings.CONDA_ENVIRONMENT_FILE
project_dir = settings.SRC_DIR
pytorch_env = Environment.from_conda_specification(env_name, file_path =str(environment_file))
pytorch_env.register(workspace=ws)
build = pytorch_env.build(workspace=ws)
pytorch_env.environment_variables["RSLEX_DIRECT_VOLUME_MOUNT"] = "True"

# Run Configuration
run_config = RunConfiguration()
run_config.environment_variables["RSLEX_DIRECT_VOLUME_MOUNT"] = "True"
run_config.environment = pytorch_env
run_config.docker = DockerConfiguration(use_docker=True, shm_size="100G")

# Note: COMMON RUNTIME has a bug where print statements sometimes, disappear.
# Set this flag as a workaround to use the legacy runtime.
run_config.environment_variables["AZUREML_COMPUTE_USE_COMMON_RUNTIME"] = "false"

# Input Dataset
dataset = Dataset.get_by_name(ws, name=input_dataset_name)

args = {
'run_azure': 1,
 'model': model_name,
 'data_folder': dataset.as_named_input('dataset').as_mount(),
 'progress_bar_refresh_rate': 25,
 'log_every_n_steps': 25,
 'flush_logs_every_n_steps': 25,
 'batch_size': 32,
 'accelerator': 'ddp',
 'output_dir': './outputs/',
 'dataset': input_dataset_name,
 }

config = ScriptRunConfig(
    source_directory = str(project_dir), 
    script = "scripts/finetune/score.py",
    arguments=helpers.argsdict2list(args),
)

for param, value in args.items():
    print(f" {param}: {value}")

run_config.target = compute_target
config.run_config = run_config

run = exp.submit(config)
display(Markdown(f"""
- Experiment: [{run.experiment.name}]({run.experiment.get_portal_url()})
- Run: [{run.display_name}]({run.get_portal_url()})
- Target: {config.run_config.target}
"""))

 run_azure: 1
 model: classifier-padchest-trained-chx-frontal-lime_train_07pvtv2y-047.ckpt
 data_folder: <azureml.data.dataset_consumption_config.DatasetConsumptionConfig object at 0x0000028DF3B96048>
 progress_bar_refresh_rate: 25
 log_every_n_steps: 25
 flush_logs_every_n_steps: 25
 batch_size: 32
 accelerator: ddp
 output_dir: ./outputs/
 dataset: padchest



- Experiment: [generate-data-score](https://ml.azure.com/experiments/generate-data-score?wsid=/subscriptions/9ca8df1a-bf40-49c6-a13f-66b72a85f43c/resourcegroups/MLOps-Prototype/workspaces/MLOps_shared&tid=72f988bf-86f1-41af-91ab-2d7cd011db47)
- Run: [nifty_lion_03mp30bz](https://ml.azure.com/runs/generate-data-score_1644445839_0bffda04?wsid=/subscriptions/9ca8df1a-bf40-49c6-a13f-66b72a85f43c/resourcegroups/MLOps-Prototype/workspaces/MLOps_shared&tid=72f988bf-86f1-41af-91ab-2d7cd011db47)
- Target: nc24-uswest2
